In [1]:
import cv2
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import xml.etree.ElementTree as ET
from torch.utils.data import Dataset, DataLoader
import os
from sklearn.model_selection import train_test_split
import torch

In [2]:
os.chdir(r"C:\Users\Lucas\Desktop\vacantion classes\Yolo project\classes")
from CNN_pytorch_ver import YoloDataset, YoloLoss, YOLOv1 #import my class CNN
from CNN import CNN_model

In [3]:
os.chdir(r"C:\Users\Lucas\Desktop\vacantion classes\Yolo project\data\Pascal_VOC\train")

In [4]:
path = r"C:\Users\Lucas\Desktop\vacantion classes\Yolo project\data\Pascal_VOC\train"

xml_list = []
for filename in os.listdir(path):
    if not filename.endswith('.xml'): continue
    fullname = os.path.join(path, filename)
    xml_list.append(fullname)

In [5]:
my_CNN = CNN_model()
my_CNN.get_annotation(xml_list)
my_CNN.encode_pictures()

In [6]:
os.chdir(r"C:\Users\Lucas\Desktop\vacantion classes\Yolo project")
my_CNN.df.to_json(r"data\annot_df.json",orient="records")
torch.save(my_CNN.encoded_picture_annot, r"data\encoded_picture.pt")

In [7]:
df_encoded = torch.load(r"data\encoded_picture.pt",weights_only=False)
df_encoded

,file_name,encoded_grid
1,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
2,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
3,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
4,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
5,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
...,...,...
13295,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
13296,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
13297,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
13298,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."


In [8]:
X_train, X_test, y_train, y_test = train_test_split(df_encoded['file_name'],df_encoded['encoded_grid'],test_size=0.8)

X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [9]:
batch_size = 16

train_dataset = YoloDataset(X_train, y_train)
val_dataset = YoloDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.backends.cudnn.benchmark = True

model = YOLOv1(S=7, B=2, C=len(my_CNN.classes)).to(device)
criterion = YoloLoss(S=7, B=2, C=len(my_CNN.classes), l_coord=5, l_noobj=0.01)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=5e-4)

scaler = torch.cuda.amp.GradScaler()

epochs = 50
for epoch in range(epochs):
    model.train()
    total_loss = 0.0

    for i, (imgs, labels) in enumerate(train_loader):
        if i % 10 == 0:
            print(f"Batch {i}/{len(train_loader)}")

        imgs, labels = imgs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            preds = model(imgs)
            loss = criterion(preds, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {avg_train_loss:.4f}")

    # Optional validation every few epochs
    if (epoch + 1) % 5 == 0:
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
                with torch.cuda.amp.autocast():
                    preds = model(imgs)
                    loss = criterion(preds, labels)
                val_loss += loss.item()
        print(f"Validation Loss: {val_loss / len(val_loader):.4f}")


C:\Users\Lucas\AppData\Local\Temp\ipykernel_11000\1759946539.py:8: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
C:\Users\Lucas\Desktop\vacantion classes\Yolo project\classes\CNN_pytorch_ver.py:210: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.Y_tensor[idx], dtype=torch.float32)


Batch 0/167


C:\Users\Lucas\AppData\Local\Temp\ipykernel_11000\1759946539.py:22: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


AttributeError: 'YOLOv1' object has no attribute 'features'

In [ ]:
#torch.save(model.state_dict(), "model/yolov1_weights.pth")
torch.save(model.state_dict(), "model/yolov1_weights2.pth")

In [ ]:
model = YOLOv1(S=7, B=2, C=len(my_CNN.classes))
#model.load_state_dict(torch.load("model/yolov1_weights.pth"))
model.load_state_dict(torch.load("model/yolov1_weights2.pth"))
model.to(device)
model.eval()

YOLOv1(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 128, kernel_size=(1, 1), stride=(1, 1))
    (7): ReLU()
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (11): ReLU()
    (12): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (16): ReLU()
    (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (18): ReLU()
    (19): Con

In [ ]:
import torch
import numpy as np
import cv2

def display_predict(tensor, S, B, C, img_path, classes, score_threshold=0.3):
    """
    tensor: PyTorch tensor of shape (S, S, B*5 + C) for a single image
    img_path: path to the image
    classes: list of class names
    """
    # Ensure tensor is on CPU and NumPy
    if isinstance(tensor, torch.Tensor):
        tensor = tensor.detach().cpu().numpy()
    
    boxes_list = [] 
    for i in range(S):
        for j in range(S):
            boxes = tensor[i,j, :B*5]           # bbox info: confidence + x + y + w + h
            class_probs = tensor[i,j, B*5:]     # class probabilities
            class_probs = 1 / (1 + np.exp(-class_probs)) # sigmoid to [0,1]

            for b in range(B):
                cls, x, y, w, h = boxes[b*5:(b+1)*5]
                cls = 1 / (1 + np.exp(-cls))  # sigmoid confidence

                img_h, img_w = 448, 448
                x_img = (j + x) * img_w / S
                y_img = (i + y) * img_h / S
                w_img = w * img_w
                h_img = abs(h) * img_h

                class_id = np.argmax(class_probs)
                class_prob = class_probs[class_id]
                final_score = cls * class_prob

                boxes_list.append({
                    'cell': (i, j),
                    'bbox': (x_img, y_img, w_img, h_img),
                    'class_id': class_id,
                    'score': final_score
                })

    # Filter by score threshold
    boxes_list = [b for b in boxes_list if b['score'] > score_threshold]

    # Load image
    img = cv2.imread(img_path)
    img = cv2.resize(img, (448, 448))

    for b in boxes_list:
        x, y, w, h = b['bbox']
        class_id = b['class_id']
        score = b['score']
        print(score)
        # Draw bounding box
        cv2.rectangle(img, (int(x - w/2), int(y - h/2)), (int(x + w/2), int(y + h/2)), (0,255,0), 2)
        cv2.putText(img, f'{classes[class_id]}:{score:.2f}', (int(x-w/2), int(y-h/2)-5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)

    cv2.imshow('YOLO prediction', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
model.eval()
with torch.no_grad():            
    img = cv2.imread(X_test[12])
    img = cv2.resize(img, (448, 448))
    img = img[..., ::-1]  # BGR to RGB
    img = img / 255.0
    sample_img = torch.tensor(img, dtype=torch.float32).permute(2, 0, 1)  # (C,H,W)
    sample_img_tensor = sample_img.unsqueeze(0).to(device)  # add batch dimension
    pred = model(sample_img_tensor)             # shape: (1, S, S, B*5 + C)
    pred = pred[0]    
    print(pred)                          # remove batch dimension
    display_predict(pred, S=7, B=2, C=len(my_CNN.classes), img_path=X_test[24], classes=my_CNN.classes, score_threshold=0.43)

tensor([[[ 3.2059e-01,  5.3304e-01,  7.2433e-01,  ..., -5.6129e-03,
           1.3125e-01, -3.4413e-02],
         [ 4.3855e-01,  5.0437e-01,  6.7549e-01,  ..., -1.1587e-02,
           1.4092e-01,  1.0947e-02],
         [ 4.5433e-01,  5.1069e-01,  7.0333e-01,  ..., -4.1800e-04,
           8.1996e-02,  6.3612e-03],
         ...,
         [ 3.6206e-01,  4.7363e-01,  6.5686e-01,  ..., -1.8519e-02,
           8.1467e-02, -2.8356e-03],
         [ 3.8179e-01,  5.4710e-01,  6.3997e-01,  ..., -6.0231e-03,
           3.1887e-02, -4.9313e-03],
         [ 4.0760e-01,  5.2286e-01,  6.7237e-01,  ..., -4.4671e-03,
           8.0058e-02,  2.8121e-03]],

        [[ 5.8473e-01,  5.6894e-01,  5.1745e-01,  ...,  9.7101e-03,
           2.7937e-02, -4.2583e-03],
         [ 5.9542e-01,  5.0293e-01,  5.4507e-01,  ..., -1.9378e-03,
           9.4919e-02, -3.0420e-03],
         [ 6.5462e-01,  4.5947e-01,  6.1109e-01,  ..., -7.3028e-04,
           9.7591e-02,  4.0041e-02],
         ...,
         [ 6.2964e-01,  5